In [1]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import tensorflow as tf

from tqdm import tqdm_notebook
from scipy.spatial.distance import hamming, cosine

%matplotlib inline

In [2]:
def image_load(path, image_size):
    """
    Load image

    :param path: String, path to image
    :image_size: tuple, size of output image
    """
    
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, image_size, cv2.INTER_CUBIC)
    return image

In [3]:
def data_preprocessing(data_path, labels_path, image_size, image_path_pickle):
    """
    Load image and labels

    :param data_path: String, path to train and test data
    :param labels_path: String, path to label
    :param image_size: tuple, single imaze size
    :param image_path_pickle: String, name of a pickle file where all image will be saved
    """
    with open(labels_path) as f:
        classes = f.read().split("\n")[: -1]

    images = []
    labels = []
    # path to pickle
    image_paths = []
    
    for image_name in os.listdir(data_path):
        try:

                # create full path to train data
                image_path = os.path.join(data_path, image_name)
                images.append(image_load(image_path, image_size))
                image_paths.append(image_path)
                for idx in range(len(classes)):
                    if classes[idx] in image_name:
                        labels.append(idx)
        except:
                pass
    
    with open(image_path_pickle + ".pickle", "wb") as f:
        pickle.dump(image_path, f)

    assert len(images) == len(labels)
    return np.array(images), np.array(labels)

In [4]:
images, labels = data_preprocessing("dataset/train", "dataset/labels.txt", (32, 32), "training_image_pickle")

In [6]:
images.shape, labels.shape

((50000, 32, 32, 3), (50000,))

In [7]:
def cosine_distance(training_set_vector, query_vector, top_n=50):
    """
    calculate cosine distance between query image and all trianing set image
    
    :param training_set_vector: numpy Matrix, vectors for all images in training set
    :param query_vector: numpy vector, query image(new image)
    :param top_n: interger, number of closest image to return
    """
    
    distances = []
    
    for i in range(len(training_set_vector)): # for train dataset 50k image
        distance.append(cosine(training_set_vector[i], query_vector[0]))
    
    return np.argsort(distance)[:top_n]
        

In [8]:
def haming_distance(training_set_vector, query_vector, top_n=50):
    """
    calculate haming distance between query image and all training images
    
    :param training_set_vector: numpy Matrix, vectors for all images in training set
    :param query_vector: numpy vector, query image(new image)
    :param top_n: interger, number of closest image to return
    """
    
    distance = []
    
    for i in range(len(training_set_vector)):
        distance.append(hamming(training_set_vector[i], query_vector[0]))
    
    return np.argsort(distance)[:top_n]

In [9]:
def cal_accuracy(y_true, y_pred):
    """
    calculate accuracy of model on softmax outputs
    
    :param y_true: numpy array, true labels of each sample
    :param y_pred: numpy matrix, softmax probabilities
    """
    
    assert len(y_true) == len(y_pred)
    
    correct += 1
    
    for i in range(len(y_true)):
        if np.argnax(y_pred)[i] == y_true[i]:
                correct += 1
    
    return correct / len(y_true)

In [33]:
class Conv(tf.keras.layers.Layer):
    
    def __init__(self, number_of_filter, kernel_size, stride=(1, 1),
                padding="SAME", activation="tf.nn.relu",
                max_pool=True, batch_norm=True):
        
        """
        define convolutional block layer

        :param number_of_filter: interger, number of filter
        :param kernel_size: tuple, size of conv layer kernel
        :param padding: String, type of padding SAME or VALID
        :param activation: tf.object, activation functuin used on the layer
        :param max_pool: boolean, true conv layer use max pooling
        :param batch_norm: boolean, true conv layer use batch normalization
        """

        super(Conv, self).__init__()

        self.conv_layer = tf.keras.layers.Conv2D(filters=number_of_filter,
                                                                                    kernel_size=kernel_size,
                                                                                    strides=stride,
                                                                                    padding=padding,
                                                                                    activation=activation)

        self.max_pool = max_pool
        if max_pool:
            self.max_pool_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2),
                                                                                                  strides=(2, 2),
                                                                                                  padding="SAME")

        self.batch_norm = batch_norm
        if batch_norm:
            self.batch_norm_layer = tf.keras.layers.BatchNormalization()

    def call(self, inputs, training):
        conv_features = x = self.conv_layer(inputs)
        if self.max_pool:
            x = self.max_pool_layer(x)
        if self.batch_norm:
            x = self.batch_norm_layer(x, training)
        
        return x, conv_features

In [37]:
class Dense(tf.keras.layers.Layer):
    def __init__(self, units, activation=tf.nn.relu, dropout=None, batch_norm=True):
        """
        define Dense layer
        
        :param units: interger, number of neurons
        :param activation: tf.object, activation functuin used on the layer
        :param dropout: dropout rate
        :param batch_norm: boolean, true conv layer use batch normalization
        """
        
        super(Dense, self).__init__()
        
        self.dense_layer = tf.keras.layers.Dense(units, activation=activation)
        
        self.dropout = dropout
        if dropout is not None:
            self.dropout_layer = tf.keras.layers.Dropout(dropout)
        
        self.batch_norm = batch_norm
        if batch_norm:
            self.batch_norm_layer = tf.keras.layers.BatchNormalization()
    
    def call(self, inputs, training):
        dense_feature = x = self.dense_layer(inputs)
        if self.dropout is not None:
            x = self.dropout_layer(x, training)
        if self.batch_norm:
            x = self.batch_norm_layer(x, training)
        return x, dense_feature

In [38]:
class BuildModel(tf.keras.Model):
    
    def __init__(self, dropout, image_size, number_of_calsses=10):
        """
        Build a model for image search
        
       :param dropout: dropout rate
       :param image_size: tuple, (height, width)
       :param number_of_calsses: integer, number of classes
       """
        
        super(BuildModel, self).__init__()
        
        self.batch_normalize_layer = tf.keras.layers.BatchNormalization()
        
        self.conv_1 = Conv(number_of_filter=64,
                                        kernel_size=(3, 3),
                                        stride=(1, 1),
                                        padding="SAME",
                                        activation=tf.nn.relu,
                                        max_pool=True,
                                        batch_norm=True)
        
        self.conv_2 = Conv(number_of_filter=128,
                                        kernel_size=(3, 3),
                                        stride=(1, 1),
                                        padding="SAME",
                                        activation=tf.nn.relu,
                                        max_pool=True,
                                        batch_norm=True)
        
        self.conv_3 = Conv(number_of_filter=256,
                                        kernel_size=(5, 5),
                                        stride=(1, 1),
                                        padding="SAME",
                                        activation=tf.nn.relu,
                                        max_pool=True,
                                        batch_norm=True)
        
        self.conv_3 = Conv(number_of_filter=512,
                                        kernel_size=(5, 5),
                                        stride=(1, 1),
                                        padding="SAME",
                                        activation=tf.nn.relu,
                                        max_pool=True,
                                        batch_norm=True)

        self.flatten_layer = tf.keras.layers.Flatten()
        
        self.dense_1 = Dense(units=128,
                                            activation=tf.nn.relu,
                                            dropout=dropout,
                                            batch_norm=True)

        self.dense_2 = Dense(units=256,
                                            activation=tf.nn.relu,
                                            dropout=dropout,
                                            batch_norm=True)

        self.dense_3 = Dense(units=512,
                                            activation=tf.nn.relu,
                                            dropout=dropout,
                                            batch_norm=True)

        self.dense_4 = Dense(units=1024,
                                            activation=tf.nn.relu,
                                            dropout=dropout,
                                            batch_norm=True)
        
        self.final_dense = tf.keras.layers.Dense(units=number_of_calsses,
                                                                        activation=None)
        
        self.final_siftmax = tf.keras.layers.Softmax()
    
    def call(self, inputs, training):
        x = self.batch_normalize_layer(inputs, training)
        x, conv1 = self.conv_1(x, training)
        x, conv2 = self.conv_2(x, training)
        x, conv3 = self.conv_3(x, training)
        x, conv4 = self.conv_4(x, training)
        
        x = self.flatten_layer(x)
        
        x, dense1 = self.dense_1(x, training)
        x, dense2 = self.dense_2(x, training)
        x, dense3 = self.dense_3(x, training)
        x, dense4 = self.dense_4(x, training)
        
        x = self.final_dense(x)
        output = self.final_siftmax(x)
        
        return output, dense2, dense4

In [39]:
model = BuildModel(0.5, (32, 32), 10)